## Introduction

In [2]:
%%capture
%load_ext sql
%sql sqlite:///../Datasets/factbook.db

For this exercise, we will be using [this database](https://dsserver-prod-resources-1.s3.amazonaws.com/257/factbook.db) from the [CIA World Factbook](https://www.cia.gov/the-world-factbook/).

## Overview of the Data

Let's start by taking a look at the data.

In [3]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///../Datasets/factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [4]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

 * sqlite:///../Datasets/factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


## Analyzing the Data

First, let's look at the minimum and maximum population and population growth in the data set.

In [8]:
%%sql
SELECT MIN(population) AS min_pop, 
MAX(population) AS max_pop, 
MIN(population_growth) AS min_pop_growth, 
MAX(population_growth) AS max_pop_growth
FROM facts;

 * sqlite:///../Datasets/factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


Well, it seems the min and max population are a bit strange--zero people and over 7 billion. Let's see what those countries are.

In [12]:
%%sql
SELECT name, population
FROM facts
WHERE population IS NOT NULL
ORDER BY population
LIMIT 5;

 * sqlite:///../Datasets/factbook.db
Done.


name,population
Antarctica,0
Pitcairn Islands,48
Cocos (Keeling) Islands,596
Holy See (Vatican City),842
Niue,1190


In [11]:
%%sql
SELECT name, population
FROM facts
ORDER BY population DESC
LIMIT 5;

 * sqlite:///../Datasets/factbook.db
Done.


name,population
World,7256490011
China,1367485388
India,1251695584
European Union,513949445
United States,321368864


The minimum is Antarctica with 0 people, and the maximum is the whole world, with a population over 7 billion. We probably won't want those skewing our summary stats. So...

What are the min and max population and growth, removing Antarctica and the World from the country list?

In [15]:
%%sql
SELECT MIN(population) AS min_pop, 
MAX(population) AS max_pop, 
MIN(population_growth) AS min_pop_growth, 
MAX(population_growth) AS max_pop_growth
FROM facts
WHERE population
BETWEEN 1 AND 7000000000;

 * sqlite:///../Datasets/factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
48,1367485388,0.0,4.02


What is the average population and area of the countries in the database?

In [16]:
%%sql
SELECT AVG(population) AS avg_pop, 
AVG(area) AS avg_area
FROM facts
WHERE population
BETWEEN 1 AND 7000000000;

 * sqlite:///../Datasets/factbook.db
Done.


avg_pop,avg_area
32377011.0125,582949.8523206752


Which countries have above average populations and below average area?

In [19]:
%%sql
SELECT name, population, area
    FROM facts
    WHERE (population > (SELECT AVG(population)
    FROM facts
    WHERE population
    BETWEEN 1 AND 7000000000))
    AND (area < (SELECT AVG(area)
    FROM facts
    WHERE population
    BETWEEN 1 AND 7000000000)
    )
    ORDER BY population;

 * sqlite:///../Datasets/factbook.db
Done.


name,population,area
Morocco,33322699,446550
Iraq,37056169,438317
Uganda,37101745,241038
Poland,38562189,312685
Kenya,45925301,580367
Spain,48146134,505370
"Korea, South",49115196,99720
Italy,61855120,301340
United Kingdom,64088222,243610
Thailand,67976405,513120


Which country has the most people?

In [23]:
%%sql
SELECT name, population
FROM facts
WHERE population
BETWEEN 1 AND 7000000000
ORDER BY population DESC
LIMIT 1;

 * sqlite:///../Datasets/factbook.db
Done.


name,population
China,1367485388


China has the most people. No surprise there. 

Which country has the highest growth rate?

In [24]:
%%sql
SELECT name, population_growth
FROM facts
WHERE population
BETWEEN 1 AND 7000000000
ORDER BY population_growth DESC
LIMIT 1;

 * sqlite:///../Datasets/factbook.db
Done.


name,population_growth
South Sudan,4.02


South Sudan. Interesting!

Which countries have the highest ratios of water to land? 

In [30]:
%%sql
SELECT name, 
    CAST(area_water AS Float) / CAST(area_land AS FLOAT) AS water_land_ratio
    FROM facts
    WHERE population
    BETWEEN 1 AND 7000000000
    ORDER BY water_land_ratio DESC
    LIMIT 5;

 * sqlite:///../Datasets/factbook.db
Done.


name,water_land_ratio
Virgin Islands,4.520231213872832
Puerto Rico,0.5547914317925592
"Bahamas, The",0.3866133866133866
Guinea-Bissau,0.2846728307254623
Malawi,0.25939625850340137


Which countries have more water than land?

In [31]:
%%sql
SELECT name, area_land, area_water
    FROM facts
    WHERE population
    BETWEEN 1 AND 7000000000
    AND area_water > area_land
    ORDER BY area_water DESC
    LIMIT 5;

 * sqlite:///../Datasets/factbook.db
Done.


name,area_land,area_water
Virgin Islands,346,1564


Only the Virgin Islands!

Which countries will add the most people to their populations next year?

In [32]:
%%sql
SELECT name, 
    CAST(birth_rate AS FLOAT) - CAST(death_rate AS FLOAT) + CAST(migration_rate AS FLOAT) AS pop_increase
    FROM facts
    WHERE population
    BETWEEN 1 AND 7000000000
    ORDER BY pop_increase DESC
    LIMIT 5;

 * sqlite:///../Datasets/factbook.db
Done.


name,pop_increase
South Sudan,40.199999999999996
American Samoa,39.269999999999996
Syria,37.96
"Micronesia, Federated States of",37.239999999999995
Tonga,35.989999999999995


Which countries have a higher death rate than birth rate?

In [33]:
%%sql
SELECT name, birth_rate, death_rate
    FROM facts
    WHERE population
    BETWEEN 1 AND 7000000000
    AND death_rate > birth_rate
    ORDER BY death_rate DESC
    LIMIT 5;

 * sqlite:///../Datasets/factbook.db
Done.


name,birth_rate,death_rate
Ukraine,10.72,14.46
Bulgaria,8.92,14.44
Latvia,10.0,14.31
Lithuania,10.1,14.27
Russia,11.6,13.69


Which countries have the highest population/area ratio, and how does it compare to list of countries with greater than average population and less than average area?

In [35]:
%%sql
SELECT name, population, area,
    CAST(population AS FLOAT) / CAST(area AS FLOAT) AS pop_per_sq_km
    FROM facts
    WHERE population
    BETWEEN 1 AND 7000000000
    ORDER BY pop_per_sq_km DESC
    LIMIT 5;

 * sqlite:///../Datasets/factbook.db
Done.


name,population,area,pop_per_sq_km
Macau,592731,28,21168.964285714286
Monaco,30535,2,15267.5
Singapore,5674472,697,8141.279770444763
Hong Kong,7141106,1108,6445.041516245487
Gaza Strip,1869055,360,5191.819444444444


Interestingly, there is no overlap with that previous list. This is because the ratio doesn't depend on having an extremely high population, only a small area (the denominator). All of these countries have a lower than average population.